In [1]:
import sys
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from scipy.stats import qmc
import itertools
from itertools import combinations_with_replacement, combinations, permutations

import bo_methods_lib
# from bo_methods_lib.bo_methods_lib.bo_functions_generic import gen_theta_set, clean_1D_arrays
from bo_methods_lib.bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Class_fxns import * #Fix this later
from bo_methods_lib.bo_methods_lib.analyze_data import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Classes_plotters import * #Fix this later
import pympler
import pickle

from pympler import asizeof

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
#Get all data from ep experiemnts
date_time_str = "2023/09/05/"
save_csv = False
name_cs_str = "CS1"      
meth_name_str_list = ["A1", "B1", "A2", "B2", "C2"]
# study_param_list = ["CONSTANT"]
study_param_list = ["CONSTANT", "DECAY", "BOYLE", "JASRASARIA"]
study_id = "ep"

#Get all ep_data  
df, theta_true = get_study_data_org(date_time_str, name_cs_str, meth_name_str_list, study_id, 
                                    study_param_list, save_csv = False)

In [7]:
#Get Best Data from ep experiment
save_csv = False
df_best = get_best_data(df, study_id, name_cs_str, theta_true, date_time_str, save_csv)
df_best

,Run Number,BO Iter,Best Error,Exploration Bias,Max EI,Theta Max EI,Min Obj,Min Obj Act,Theta Min Obj,Min Obj Cum.,Theta Min Obj Cum.,Time/Iter,EP Method Val,BO Method,Max Evals,Total Run Time,L2 Norm Theta
20,10,0,5.342819,1,0.0,"[0.3157894736842106, -0.3157894736842106]",0.836632,[0.08118771297266594],"[1.0650997052362372, -0.9662152154181552]",[0.08118771297266594],"[1.0650997052362372, -0.9662152154181552]",0.241593,CONSTANT,A1,2,0.483186,0.073344
754,10,17,-2.007167,0.83,0.000007,"[-2.0, -0.10526315789473695]",0.113304,[0.011270556801092754],"[0.9976258526369931, -1.0181611989790127]",[0.011270556801092754],"[0.9976258526369931, -1.0181611989790127]",0.10183,DECAY,B1,21,2.13843,0.018316
981,9,4,0.000007,1,0.0,"[-0.5263157894736843, 1.1578947368421053]",0.0,[1.5020460791557627e-06],"[1.0001577450275225, -0.9998080126274008]",[1.5020460791557627e-06],"[1.0001577450275225, -0.9998080126274008]",0.679958,CONSTANT,A2,5,3.399789,0.000248
2273,9,33,0.0,1,52.090981,"[1.000015606064711, -0.9999971207707669]",0.0,[4.854498154158254e-09],"[1.0000218957117102, -0.9999986685231427]",[4.854498154158254e-09],"[1.0000218957117102, -0.9999986685231427]",112.130124,CONSTANT,B2,34,3812.424203,0.000022
4253,10,7,0.0,55977744.874759,0.0,"[0.5263157894736841, 0.10526315789473673]",0.0,[7.33527449102052e-08],"[1.0000158775181107, -1.0000456430489189]",[7.33527449102052e-08],"[1.0000158775181107, -1.0000456430489189]",1.145314,BOYLE,C2,9,10.307826,0.000048


In [8]:
#Get Median Data from ep experiment
save_csv = False
df_median = get_median_data(df, study_id, name_cs_str, theta_true, date_time_str, save_csv)

In [9]:
#Get Median of Best Data from ep experiment
save_csv = False
df_median_best = get_median_data(df_best, study_id, name_cs_str, theta_true, date_time_str, save_csv)
df_median_best

,Run Number,BO Iter,Best Error,Exploration Bias,Max EI,Theta Max EI,Min Obj,Min Obj Act,Theta Min Obj,Min Obj Cum.,Theta Min Obj Cum.,Time/Iter,EP Method Val,BO Method,Max Evals,Total Run Time,L2 Norm Theta
20,10,0,5.342819,1,0.0,"[0.3157894736842106, -0.3157894736842106]",0.836632,[0.08118771297266594],"[1.0650997052362372, -0.9662152154181552]",[0.08118771297266594],"[1.0650997052362372, -0.9662152154181552]",0.241593,CONSTANT,A1,2,0.483186,0.073344
754,10,17,-2.007167,0.83,0.000007,"[-2.0, -0.10526315789473695]",0.113304,[0.011270556801092754],"[0.9976258526369931, -1.0181611989790127]",[0.011270556801092754],"[0.9976258526369931, -1.0181611989790127]",0.10183,DECAY,B1,21,2.13843,0.018316
981,9,4,0.000007,1,0.0,"[-0.5263157894736843, 1.1578947368421053]",0.0,[1.5020460791557627e-06],"[1.0001577450275225, -0.9998080126274008]",[1.5020460791557627e-06],"[1.0001577450275225, -0.9998080126274008]",0.679958,CONSTANT,A2,5,3.399789,0.000248
2273,9,33,0.0,1,52.090981,"[1.000015606064711, -0.9999971207707669]",0.0,[4.854498154158254e-09],"[1.0000218957117102, -0.9999986685231427]",[4.854498154158254e-09],"[1.0000218957117102, -0.9999986685231427]",112.130124,CONSTANT,B2,34,3812.424203,0.000022
4253,10,7,0.0,55977744.874759,0.0,"[0.5263157894736841, 0.10526315789473673]",0.0,[7.33527449102052e-08],"[1.0000158775181107, -1.0000456430489189]",[7.33527449102052e-08],"[1.0000158775181107, -1.0000456430489189]",1.145314,BOYLE,C2,9,10.307826,0.000048


In [10]:
#Get all data from sf experiemnts
date_time_str = "2023/09/06/"
# date_time_str = "No_Date/"
save_csv = False
name_cs_str = "CS1" 
meth_name_str_list = ["A1", "B1", "A2", "B2", "C2"]
# meth_name_str_list = ["C2"]
f = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
# f = [0.9]
precision = 1
study_param_list = [f"{number:.{precision}f}" for number in f]
study_id = "sf"

#Get all sf_data  
df, theta_true = get_study_data_org(date_time_str, name_cs_str, meth_name_str_list, study_id, 
                                    study_param_list, save_csv = False)

In [11]:
#Get Best Data from ep experiment
df_best = get_best_data(df, study_id, name_cs_str, theta_true, date_time_str, save_csv)
df_best

,Run Number,BO Iter,Best Error,Exploration Bias,Max EI,Theta Max EI,Min Obj,Min Obj Act,Theta Min Obj,Min Obj Cum.,Theta Min Obj Cum.,Time/Iter,Sep Fact,BO Method,Max Evals,Total Run Time,L2 Norm Theta
65,2,1,1.550589,0.0,0.0,"[0.5263157894736841, 0.7368421052631575]",-0.211862,[0.0138825780137719],"[1.007264742277993, -1.0198188956258143]",[0.0138825780137719],"[1.007264742277993, -1.0198188956258143]",0.117507,0.7,A1,2,0.235014,0.021108
379,12,4,0.438635,0.96,0.381856,"[0.9852207221004373, -1.017601621618769]",1.040027,[0.01266044550339453],"[0.9853130537230843, -1.0175762051665986]",[0.01266044550339453],"[0.9853130537230843, -1.0175762051665986]",0.16592,0.5,B1,7,1.161442,0.022905
1136,9,2,0.000032,1,0.000005,"[-0.5263157894736843, -0.3157894736842106]",0.000011,[1.2985206090679695e-05],"[0.998984534334488, -0.9997195853431529]",[1.2985206090679695e-05],"[0.998984534334488, -0.9997195853431529]",1.060315,0.9,A2,4,4.24126,0.001053
3957,11,35,0.0,1,52.113367,"[0.9999320773112595, -0.9999792467583222]",0.0,[6.940996831652715e-08],"[0.9999252883337223, -0.9999800061498159]",[6.940996831652715e-08],"[0.9999252883337223, -0.9999800061498159]",63.639614,0.9,B2,36,2291.026099,0.000077
4711,8,37,0.0,2151972563.222417,5232.881352,"[0.9998958408004562, -1.0000110523070578]",0.0,[1.240680225828881e-07],"[0.9998914398175471, -1.000013520018318]",[1.240680225828881e-07],"[0.9998914398175471, -1.000013520018318]",3.739789,0.5,C2,38,142.112,0.000109


In [12]:
#Get Median Data from ep experiment
df_median = get_median_data(df, study_id, name_cs_str, theta_true, date_time_str, save_csv)

In [13]:
#Get Median of Best Data from ep experiment
df_median_best = get_median_data(df_best, study_id, name_cs_str, theta_true, date_time_str, save_csv)
df_median_best

,Run Number,BO Iter,Best Error,Exploration Bias,Max EI,Theta Max EI,Min Obj,Min Obj Act,Theta Min Obj,Min Obj Cum.,Theta Min Obj Cum.,Time/Iter,Sep Fact,BO Method,Max Evals,Total Run Time,L2 Norm Theta
65,2,1,1.550589,0.0,0.0,"[0.5263157894736841, 0.7368421052631575]",-0.211862,[0.0138825780137719],"[1.007264742277993, -1.0198188956258143]",[0.0138825780137719],"[1.007264742277993, -1.0198188956258143]",0.117507,0.7,A1,2,0.235014,0.021108
379,12,4,0.438635,0.96,0.381856,"[0.9852207221004373, -1.017601621618769]",1.040027,[0.01266044550339453],"[0.9853130537230843, -1.0175762051665986]",[0.01266044550339453],"[0.9853130537230843, -1.0175762051665986]",0.16592,0.5,B1,7,1.161442,0.022905
1136,9,2,0.000032,1,0.000005,"[-0.5263157894736843, -0.3157894736842106]",0.000011,[1.2985206090679695e-05],"[0.998984534334488, -0.9997195853431529]",[1.2985206090679695e-05],"[0.998984534334488, -0.9997195853431529]",1.060315,0.9,A2,4,4.24126,0.001053
3957,11,35,0.0,1,52.113367,"[0.9999320773112595, -0.9999792467583222]",0.0,[6.940996831652715e-08],"[0.9999252883337223, -0.9999800061498159]",[6.940996831652715e-08],"[0.9999252883337223, -0.9999800061498159]",63.639614,0.9,B2,36,2291.026099,0.000077
4711,8,37,0.0,2151972563.222417,5232.881352,"[0.9998958408004562, -1.0000110523070578]",0.0,[1.240680225828881e-07],"[0.9998914398175471, -1.000013520018318]",[1.240680225828881e-07],"[0.9998914398175471, -1.000013520018318]",3.739789,0.5,C2,38,142.112,0.000109
